
# Prediction Market

**Collaborators: Josefina Waugh, Anne Bastian, Bridget Smart, Ebba Mark**

Preliminary model implementation of:
- Betters engage in a market trading contracts on a binary election outcome.
- Betters are endowed with initial budget, risk aversion, belief about outcome.
- Next extension will incorporate more detail into formation of this belief.

In [93]:
# Import packages
import numpy as np
import random as random
import matplotlib.pyplot as plt

### Defining Agents, Classes, Functions

In [106]:
## Defining classes
class better:
    def __init__(self): 
                # Their current budget
                self.budget = np.random.uniform(100,1000)
                # Their personal contract valuation - this will ultimately depend on their beliefs and evolve in relation to market activity
                self.market_valuation = np.random.uniform(0,1)
                # The number of contracts currently held
                self.n_contracts = 0
                # Risk aversion score (if we indeed try to vary the risk aversion)
                self.risk_av = np.random.uniform(0,1)
                # Definition (loose): your willingness to change your opinion in the face of contradicting evidence
                # Will ideally be used to process "fact" or "news" - again, loosely defined for now
                self.stubbornness = np.random.uniform(0,1)
                # TASK: consider adding inherent "expert" value here to factor into the 
                # update_belief as the noise around the true value
                # ...
                
    ### TASK
    def exp_utility(self, mkt_price, new_c):
        #return self.market_valuation * self.utility(self.budget - mkt_price*new_c + self.n_contracts + new_c) + ((1-self.market_valuation)*self.utility(self.budget - mkt_price*new_c))
        return self.market_valuation * (self.budget - mkt_price*new_c + self.n_contracts + new_c) + ((1-self.market_valuation)*(self.budget - mkt_price*new_c))

    # testing range is the range of maximum amount of contracts that can be sold at that time
    #testing_range_x = np.arange((self.budget + n_contracts, self.budget)
    #np.argmax([b.exp_utility(m, x) for x in testing_range_x])

    ## I think something like this is probably best        
    # testing_range_x = np.arange((-y-z), y) # does this range make sense? - most you can sell is -y+z, most you can buy is y
    # testing_range_x[np.argmax([exp_utility(x, rho, p, q, y, z) for x in testing_range_x])]
    
    
    # def utility(self, w):
    #     if self.risk_av ==1:
    #         return np.log(w)
    #     else:
    #         return (w**(1-self.risk_av))/(1-self.risk_av)
        

    # # Function in which better sells of buys depending on their budget, market_valuation, and portfolio size
    # def buy(self, m):
    #     # Need to decide whether buyer buys the max of what is possible with their budget or not - depending on risk aversion?
    #     # For example: purchased_contracts = (1-risk_aversion)*np.round(self.budget/mkt_price)
    #     # Next step: include log function to acccommodate risk aversion
    #     # testing range is the range of maximum amount of contracts that can be sold at that time
    #     buy_range = np.arange(0, self.budget)
    #     new_contracts = np.argmax([self.exp_utility(m, x) for x in buy_range])
    #     print("Buy:", new_contracts)
    #     return new_contracts
    #     #self.n_contracts += new_contracts
    #     #self.capital -= new_contracts*mkt_price 

    def trade(self, m):
        c_range = np.arange((-1*(self.budget + self.n_contracts)), self.budget)
        offered_contracts = np.argmax([self.exp_utility(m, x) for x in c_range])
        print(offered_contracts)
        if offered_contracts < 0:
            print("Buy: ", offered_contracts)
        if offered_contracts > 0:
            print("Sell: ", offered_contracts)
        if offered_contracts == 0:
            print("Hold.")
        return offered_contracts
        # For example: sold_contracts = np.round((1-risk_aversion)*self.n_contracts)
        #self.n_contracts -= sold_contracts
        #self.capital += sold_contracts*mkt_price


    # Need to define a function that updates beliefs as a function of the current market price, stubbornness, (risk aversion?) and available information
    # Stubbornness for now interacts here as 1-stubbornness - could consider renaming to "openness" or "amenability" ie. antonym just for clarity.
    def update_belief(self, true_value): # stubbornness, risk aversion, information
          self.market_valuation += (1-self.stubbornness)*(np.random.normal(true_value, 0.05) - self.market_valuation)
          # ensure value is within range [0,1]
          self.market_valuation = np.clip(self.market_valuation, 0, 1)
    

### Function to initialise betters


In [95]:
# Set initial parameters
parameters = {'n_betters': 500, # The number of betting agents
              'el_outcome': 1, # Ultimate election outcome - assuming we know this to begin with and it does not change over time
              't_election': 50, # Time until election takes place (ie. time horizon of betting)
              'initial_price': 0.5} # Perhaps a function of uncertainty or time until election outcome?

### Model Run

In [107]:
def run_market(n_betters, t_election, initial_price, el_outcome):
    # Initialise betting population
    betters = [better() for _ in range(n_betters)]
    orders = []
    # Initial market price
    mkt_price = initial_price

    # Record price history over time
    price_history = [mkt_price]

    # I think we need to update demand and supply after each better bets...in that case we need to random shuffle the agents
    random.shuffle(betters)
    demand = 0
    supply = 0

    for t in range(t_election):
        for b in betters:
            # With the current utility function it should be fine to just "trade" and not distinguish between buy and sell - also probably good given non-linear utility function
            b.trade(mkt_price)
            # if (b.market_valuation >= mkt_price) & (b.budget >= 1): # Each trader proposes to sell or buy conditional on market value and their utility function
            #     buy_offers = b.buy(mkt_price)
            #     demand += buy_offers
            # elif (b.market_valuation < mkt_price): #                demand += buy_offers
            #     sell_offers = b.sell(mkt_price)
            #     supply += sell_offers


        # Fulfill buy-sell orders once every better has placed their order
        #    Traders update portfolios according to fulfilled orders or not
        ...
        # Calculate supply and demand (buy contracts - sell contracts)
        ...
        #print("Demand: ", demand)
        #print("Supply:",  supply)
        # Update market price price from above difference in d and s
        ...
        price_history.append(mkt_price)
        # (Ignore for now) Traders updated their market valuation - have not yet decided how they do that...
        #      Incorporate election outcome as random walk that changes with each time step
        #      Pass this parameter to each agent with some fuzzy noise
        for b in betters:
            b.update_belief(mkt_price)
        
    return price_history

pred_market = run_market(**parameters)


0
0
0
1374
799
0
1590
0
0
424
262
0
1217
1172
321
1112
1510
0
1183
0
386
1796
0
935
0
0
1779
1476
1252
1620
0
1083
0
0
0
1948
1657
0
0
1273
0
0
1860
0
1024
1467
221
0
0
989
1467
0
0
0
0
0
0
0
647
1694
0
720
0
1215
0
0
0
0
0
1583
0
0
0
1884
0
1709
1140
414
0
0
788
0
0
509
1140
754
0
0
1009
1373
1351
0
306
0
0
0
1552
1696
1330
0
0
0
0
379
0
0
0
0
0
424
511
1603
0
0
1549
0
1323
969
707
1643
0
1706
1137
1957
280
1024
0
317
924
1493
738
0
0
1126
0
0
1224
697
226
0
1844
0
1303
0
0
0
1856
269
462
0
966
0
1951
0
1550
1825
0
1729
0
0
551
0
1699
0
1468
0
925
1466
310
1906
1638
883
0
0
0
0
0
0
292
1397
0
1511
833
0
377
751
448
1351
0
1984
1393
629
903
0
0
1851
1680
0
0
1149
0
1654
1975
1607
0
0
222
387
693
262
0
0
933
0
467
0
227
0
1168
0
0
0
885
0
1298
1588
854
1231
1329
692
1132
0
953
1515
1159
780
0
0
668
1131
1889
463
573
0
0
1428
1035
448
0
1690
0
0
0
1502
597
0
0
561
981
1605
0
296
0
481
0
0
0
0
0
0
1560
483
0
949
1148
1824
1154
0
817
1905
0
0
829
0
638
0
1190
0
1189
0
1589
0
1704
1728
0
47

Questions:
- How do we update market valuation of a better (ie. what is the new information that they are processing)?
- Dynamic price updating versus once per discrete time step?




## Results

The following graph shows time series of CSE and voter intention (true intended voting behaviour of a population) in the top panel and the difference between the two estimates in the other.

In [ ]:



## Election Result
fig, ax = plt.subplots(2)
ax[0].plot(pred_market[:,0], pred_market[:,1], color = "blue", linestyle = "dotted")
ax[0].plot(el_outcome[:,0], el_outcome[:,2], color = "purple", linestyle = "dotted")
ax[1].plot(trade_vol[:,0], trade_vol[:,1], color = "red", linestyle = "dotted")
# Add title and axis labels
ax[0].set_title('Market Price')
ax[1].set_title('Trade Volume')
plt.xlabel('Time')
plt.ylabel('Result')
plt.xticks(rotation=45)
fig.tight_layout()

# Display the plot
plt.show()